In [2]:
import postal
import sys
import os
from datetime import datetime, timezone
from pathlib import Path
import pandas as pd
import re
from postal.expand import expand_address
from postal.parser import parse_address

In [3]:
from dotenv import load_dotenv
load_dotenv('/home/james/PDS/client_data_feeds/realestate/airflow/.env')

# Add airflow directory and parent directory to path (for pyRealtor)
sys.path.insert(0, '/home/james/PDS/client_data_feeds/realestate/airflow')
sys.path.insert(0, '/home/james/PDS/client_data_feeds/realestate')
from include.db.connections import get_master_db_connection

conn = get_master_db_connection()

In [36]:
# pull the entire properties table
properties_df = pd.read_sql("SELECT * FROM properties", conn)

/tmp/ipykernel_34350/510714797.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  properties_df = pd.read_sql("SELECT * FROM properties", conn)


In [9]:
pools_df = pd.read_sql("SELECT * FROM pools", conn)
listings_df = pd.read_sql("SELECT * FROM listings", conn)

/tmp/ipykernel_34350/118387109.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pools_df = pd.read_sql("SELECT * FROM pools", conn)
/tmp/ipykernel_34350/118387109.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  listings_df = pd.read_sql("SELECT * FROM listings", conn)


In [37]:
len(properties_df) # 23533


22315

In [8]:
properties_df.head()

,id,address_id,address_number,country,lat,lon,postal_code,street_name,municipality,province_state,geom,updated_at
0,3aa7a580-f6eb-427a-a100-9e3243d196b4,1635,289,Canada,43.135541,-80.766408,N4S 4W8,ingersoll avenue,Woodstock,Ontario,0101000020E6100000ED388FD40C3154C0C6E127695991...,2026-01-19 18:35:22.398386+00:00
1,2db2d4d8-db1a-472f-ad8e-7db7fdbe991b,1637,185,Canada,43.135138,-80.764779,N4S 6E7,vansittart avenue,Woodstock,Ontario,0101000020E6100000B96A9E23F23054C01AF7E6374C91...,2026-01-19 18:35:22.398386+00:00
2,54305c58-7ffb-4101-82a4-64d15f2c9d6b,1643,169,Canada,43.135483,-80.760431,N4S 6K2,graham street,Woodstock,Ontario,0101000020E6100000B4160BE8AA3054C0FD14C7815791...,2026-01-19 18:35:22.398386+00:00
3,a0d6fd81-0d80-4d96-9012-8970470c1575,1644,406,Canada,43.135737,-80.761312,N4S 4X5,ingersoll avenue,Woodstock,Ontario,0101000020E61000008F238C55B93054C08D6ECED25F91...,2026-01-19 18:35:22.398386+00:00
4,45588391-35c9-4efe-81e1-48717ad3825b,1649,140,Canada,43.134532,-80.759525,N4S 6J9,graham street,Woodstock,Ontario,0101000020E61000006E3F2A0F9C3054C066625F573891...,2026-01-19 18:35:22.398386+00:00


In [7]:
# apply to-lower on street_name column
properties_df['street_name'] = properties_df['street_name'].str.lower()

In [13]:
def is_within_1_km(lat1, lon1, lat2, lon2):
    from geopy.distance import geodesic
    distance = geodesic((lat1, lon1), (lat2, lon2)).km
    return distance <= 1

properties_df['full_address'] = properties_df['address_number'].astype(str) + ' ' + properties_df['street_name'] 
properties_df['full_address_expanded'] = properties_df['full_address'].apply(lambda x: expand_address(x) if expand_address(x) else x)


In [14]:
properties_df['full_address_expanded'].head()

0     [289 ingersoll avenue]
1    [185 vansittart avenue]
2        [169 graham street]
3     [406 ingersoll avenue]
4        [140 graham street]
Name: full_address_expanded, dtype: object

In [18]:
import numpy as np
import pandas as pd

EARTH_RADIUS_M = 6371000.0

def _haversine_m(lat1, lon1, lat2, lon2):
    """Vectorized haversine distance in meters."""
    lat1 = np.deg2rad(lat1); lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2); lon2 = np.deg2rad(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2.0 * EARTH_RADIUS_M * np.arcsin(np.sqrt(a))

def find_duplicate_pairs_fast(properties_df: pd.DataFrame, km: float = 1.0):
    """
    Fast replacement for the O(n^2) loop:
    - any overlap between full_address_expanded arrays
    - within `km` kilometers
    Returns list[(id_i, id_j)].
    """
    df = properties_df[['id', 'lat', 'lon', 'full_address_expanded']].copy()

    # Ensure list-like; drop empties early
    df['full_address_expanded'] = df['full_address_expanded'].apply(lambda x: x if isinstance(x, (list, tuple, set)) else [])
    df = df[df['full_address_expanded'].map(len) > 0].reset_index(drop=True)

    if df.empty:
        return []

    # Create token -> row index (explode)
    tokens = df[['id', 'full_address_expanded']].explode('full_address_expanded', ignore_index=True)
    tokens = tokens.rename(columns={'full_address_expanded': 'token'})

    # If tokens can be messy, normalize (optional):
    # tokens['token'] = tokens['token'].astype(str).str.strip().str.lower()

    # Self-join on token to generate candidate pairs (only rows sharing >=1 token)
    merged = tokens.merge(tokens, on='token', suffixes=('_a', '_b'))

    # Keep only distinct pairs (id_a < id_b)
    merged = merged[merged['id_a'] < merged['id_b']]

    # Deduplicate pairs that share multiple tokens
    pairs = merged[['id_a', 'id_b']].drop_duplicates()

    # Join coords for vectorized distance check
    coords = df.set_index('id')[['lat', 'lon']]
    pairs = pairs.join(coords, on='id_a').rename(columns={'lat': 'lat_a', 'lon': 'lon_a'})
    pairs = pairs.join(coords, on='id_b').rename(columns={'lat': 'lat_b', 'lon': 'lon_b'})

    # Fast bounding-box prefilter before haversine
    # ~1 deg lat ≈ 111.32 km; lon scale depends on latitude
    max_km = float(km)
    lat_eps = max_km / 111.32
    lat_mean = np.deg2rad((pairs['lat_a'].to_numpy() + pairs['lat_b'].to_numpy()) / 2.0)
    lon_eps = max_km / (111.32 * np.maximum(np.cos(lat_mean), 1e-12))

    lat_a = pairs['lat_a'].to_numpy()
    lon_a = pairs['lon_a'].to_numpy()
    lat_b = pairs['lat_b'].to_numpy()
    lon_b = pairs['lon_b'].to_numpy()

    bbox_ok = (np.abs(lat_a - lat_b) <= lat_eps) & (np.abs(lon_a - lon_b) <= lon_eps)

    if not np.any(bbox_ok):
        return []

    pairs2 = pairs.loc[bbox_ok].copy()

    # Vectorized haversine for the remaining candidates
    d_m = _haversine_m(
        pairs2['lat_a'].to_numpy(),
        pairs2['lon_a'].to_numpy(),
        pairs2['lat_b'].to_numpy(),
        pairs2['lon_b'].to_numpy(),
    )

    keep = d_m <= (max_km * 1000.0)
    out = list(zip(pairs2.loc[keep, 'id_a'], pairs2.loc[keep, 'id_b']))
    return out

# usage:
duplicate_pairs = find_duplicate_pairs_fast(properties_df, km=1.0)


In [19]:
len(duplicate_pairs)

1242

In [23]:
print("LISTINGS COLS:", listings_df.columns.tolist())
print("POOLS COLS:", pools_df.columns.tolist())


LISTINGS COLS: ['id', 'mls_id', 'property_address_id', 'bathrooms', 'bedrooms', 'date_collected', 'description', 'house_cat', 'is_removed', 'listing_address_number', 'price', 'removal_date', 'size_sqft', 'stories', 'created_at', 'updated_at']
POOLS COLS: ['id', 'pool_id', 'property_id', 'pool_type', 'cover_type', 'filter_type', 'created_at', 'updated_at']


In [33]:
from collections import defaultdict, deque
import pandas as pd


def build_drop_lists_from_duplicates(
    properties_df: pd.DataFrame,
    duplicate_pairs: list[tuple[int, int]],
    pools_df: pd.DataFrame,
    listings_df: pd.DataFrame,
):
    """
    Rules:
      - If exactly one property in a duplicate set has ANY listing(s): keep that property.
      - If both have listings: only dedupe if their listings are the SAME (same mls_id set).
      - If listings are the same:
          - if one has a pool and the other doesn't: keep the one WITH a pool
          - if both have pools or both don't: keep lower property id (drop higher id)

    Output:
      dict with ids/rows to drop in each table:
        - properties_drop_ids: property ids
        - pools_drop_row_ids: pools table row ids
        - listings_drop_row_ids: listings table row ids
    """

    if properties_df.empty or not duplicate_pairs:
        return {
            "properties_drop_ids": [],
            "pools_drop_row_ids": [],
            "listings_drop_row_ids": [],
        }

    # --- expected columns (based on what you printed) ---
    prop_pk = "id"
    pools_fk = "property_id"
    pools_pk = "id"
    listings_fk = "property_address_id"
    listings_pk = "id"
    listing_identity = "mls_id"

    prop_ids = set(properties_df[prop_pk].tolist())

    # --- pool presence per property ---
    pools_by_prop = pools_df.groupby(pools_fk)[pools_pk].apply(list).to_dict()
    has_pool = {pid: bool(pools_by_prop.get(pid)) for pid in prop_ids}

    # --- listing signature per property: frozenset of mls_id values ---
    # Drop NaNs so "no listing identity" doesn't accidentally create signatures
    listings_sig_by_prop = (
        listings_df.dropna(subset=[listing_identity])
        .groupby(listings_fk)[listing_identity]
        .apply(lambda s: frozenset(s.astype(str).tolist()))
        .to_dict()
    )

    def listing_sig(pid: int):
        return listings_sig_by_prop.get(pid, None)

    listing_sig_map = {pid: listing_sig(pid) for pid in prop_ids}

    # --- build duplicate graph (handles transitive duplicates) ---
    adj = defaultdict(set)
    for a, b in duplicate_pairs:
        if a in prop_ids and b in prop_ids and a != b:
            adj[a].add(b)
            adj[b].add(a)

    # --- connected components ---
    visited = set()
    components = []
    for pid in adj.keys():
        if pid in visited:
            continue
        q = deque([pid])
        visited.add(pid)
        comp = []
        while q:
            x = q.popleft()
            comp.append(x)
            for y in adj[x]:
                if y not in visited:
                    visited.add(y)
                    q.append(y)
        components.append(comp)

    # --- choose drops per component ---
    properties_to_drop = set()

    def pick_keeper(cands: list[int]) -> int:
        # Prefer pool presence, then lowest property id
        return sorted(cands, key=lambda pid: (0 if has_pool.get(pid, False) else 1, pid))[0]

    for comp in components:
        # Partition within component by listing signature
        by_sig = defaultdict(list)
        none_group = []
        for pid in comp:
            sig = listing_sig_map.get(pid)
            if sig is None:
                none_group.append(pid)
            else:
                by_sig[sig].append(pid)

        if by_sig:
            # If ANY listed property exists in the component: all non-listed lose
            for pid in none_group:
                properties_to_drop.add(pid)

            # Only dedupe among properties that share the SAME listing signature (same mls_id set)
            for sig, group in by_sig.items():
                if len(group) <= 1:
                    continue
                keeper = pick_keeper(group)
                for pid in group:
                    if pid != keeper:
                        properties_to_drop.add(pid)

        else:
            # No listings anywhere: dedupe whole component
            if len(comp) > 1:
                keeper = pick_keeper(comp)
                for pid in comp:
                    if pid != keeper:
                        properties_to_drop.add(pid)

    # --- derive row drops in pools and listings tables ---
    pools_drop_rows = pools_df.loc[pools_df[pools_fk].isin(properties_to_drop), pools_pk].tolist()
    listings_drop_rows = listings_df.loc[listings_df[listings_fk].isin(properties_to_drop), listings_pk].tolist()

    return {
        "properties_drop_ids": sorted(properties_to_drop),
        "pools_drop_row_ids": sorted(pools_drop_rows),
        "listings_drop_row_ids": sorted(listings_drop_rows),
    }


# usage:
# drops = build_drop_lists_from_duplicates(properties_df, duplicate_pairs, pools_df, listings_df)
# drops["properties_drop_ids"], drops["pools_drop_row_ids"], drops["listings_drop_row_ids"]
drops = build_drop_lists_from_duplicates(properties_df, duplicate_pairs, pools_df, listings_df)
len(drops["properties_drop_ids"]), len(drops["pools_drop_row_ids"]), len(drops["listings_drop_row_ids"])

(1218, 1201, 0)

In [34]:
def drop_duplicate_rows(conn, drops: dict):
    pool_ids = drops.get("pools_drop_row_ids", [])
    prop_ids = drops.get("properties_drop_ids", [])
    if not pool_ids and not prop_ids:
        print("Nothing to drop.")
        return

    with conn.cursor() as cur:
        try:
            # ---- delete pools first ----
            if pool_ids:
                cur.execute(
                    """
                    DELETE FROM pools
                    WHERE id = ANY(%s::uuid[])
                    """,
                    (pool_ids,),
                )
                print(f"Deleted {cur.rowcount} rows from pools")

            # ---- delete properties ----
            if prop_ids:
                cur.execute(
                    """
                    DELETE FROM properties
                    WHERE id = ANY(%s::uuid[])
                    """,
                    (prop_ids,),
                )
                print(f"Deleted {cur.rowcount} rows from properties")

            conn.commit()

        except Exception:
            conn.rollback()
            raise


In [32]:
drops

In [35]:
conn = get_master_db_connection()
drops = drop_duplicate_rows(
   conn,drops
)



Deleted 0 rows from pools
Deleted 0 rows from properties
